In [ ]:
# default_exp modelling

# Modelling & submitting

> Playing with different models and submitting predictions over the test set to kaggle. 

Current implementation of this notebook leads to (private leaderboard score): 
- RandomForest 1.65, 
- tabular_learner 1.55 and 
- lgbm 1.67. 

Those scores relate to a validation set error (`nb score`) of .8 - 1.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#export
from ashrae import preprocessing

import pandas as pd
from pathlib import Path
import os
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import typing
import pickle


from sklearn import linear_model, tree, model_selection, ensemble

from fastai.tabular.all import *

import lightgbm as lgb

import ipywidgets as widgets

In [ ]:
pd.options.plotting.backend = "plotly"

In [ ]:
do_test = False
do_submit = False
data_path = Path("../data")

In [ ]:
#export
def evaluate_torch(y_true:torch.Tensor, y_pred:torch.Tensor): return torch.sqrt(F.mse_loss(y_true, y_pred))

In [ ]:
#export
cnr = lambda x: x.clone().numpy().ravel() # clone numpy ravel

## Loading

In [ ]:
%%time
var_names = preprocessing.load_var_names(data_path/'var_names.pckl')
var_names

In [ ]:
%%time
#df = preprocessing.load_df(data_path/'X.parquet')
df = preprocessing.load_df('s3://dkb-ds-playground/hackdays/ashrae-energy-prediction/features/X.parquet')

if do_test:
    df_test = preprocessing.load_df(data_path/'X_test.parquet')

## Sampling `df`

In [ ]:
%%time
n = len(df)

if True: # per building_id and meter sampling
    n_sample_per_bid = 50
    replace = True

    df = (df.groupby(['building_id', 'meter'])
         .sample(n=n_sample_per_bid, replace=replace))

if False: # general sampling
    frac_samples = .05
    replace = False

    df = (df.sample(frac=frac_samples, replace=replace))

print(f'using {len(df)} samples = {len(df)/n*100:.2f} %')

## Split

In [ ]:
#!aws s3 ls s3://dkb-ds-playground/hackdays/ashrae-energy-prediction/raw_data/ --human-readable

In [ ]:
#!aws s3 cp 's3://dkb-ds-playground/hackdays/ashrae-energy-prediction/raw_data/split_timestamps_train.parquet' {data_path}/

In [ ]:
#!aws s3 cp 's3://dkb-ds-playground/hackdays/ashrae-energy-prediction/raw_data/split_timestamps_val.parquet' {data_path}/

In [ ]:
%%time
t_train = pd.read_parquet(data_path/'split_timestamps_train.parquet')
t_val = pd.read_parquet(data_path/'split_timestamps_val.parquet')

In [ ]:
t_train.head()

In [ ]:
t_val.head()

In [ ]:
#export
def split_dataset(X:pd.DataFrame, split_kind:str='random',
                  train_frac:float=8, t_train:pd.DataFrame=None):
    
    def random_split():
        n_train = int(len(X)*train_frac)
        train_bool = X.index.isin(np.random.choice(X.index.values, size=n_train, replace=False))
        return train_bool
    
    def time_split():
        time_col = 'timestampElapsed'
        ts = X[time_col].sort_values(ascending=True)
        ix = int(len(X)*train_frac)
        threshold_t = ts.iloc[ix:].values[0]
        return X[time_col] < threshold_t
    
    def fix_time_split():
        assert t_train is not None
        time_col = 'timestamp'
        assert time_col in X.columns
        
        mask = X[time_col].isin(t_train[time_col])
        assert mask.sum() > 0
        return mask
     
    split_funs = {
        'random': random_split,
        'time': time_split,
        'fix_time': fix_time_split,
    }
    
    assert split_kind in split_funs
    train_bool = split_funs[split_kind]()
    
    train_idx = np.where(train_bool)[0]
    valid_idx = np.where(~train_bool)[0]

    return (list(train_idx), list(valid_idx))

In [ ]:
len(t_train), len(t_val)

In [ ]:
%%time
#split_kind = 'random'
#split_kind = 'time'
split_kind = 'fix_time'
splits = split_dataset(df, split_kind=split_kind, train_frac=.8,
                       t_train=t_train)
#splits=None

In [ ]:
%%time
to = preprocessing.get_tabular_object(df, var_names, splits=splits)

In [ ]:
%%time
train_bs = 256*8
val_bs = 256*8

dls = to.dataloaders(bs=train_bs, val_bs=val_bs)

Warning: Takes about 12min with the test set

In [ ]:
%%time
test_bs = 1024*4

if do_test:
    test_dl = dls.test_dl(df_test, bs=test_bs) 

## Modelling with

### `sklearn`

In [ ]:
%%time
params = {'n_estimators': 20, 'max_features': 'sqrt'}
model = ensemble.RandomForestRegressor
# params = {}
# model = linear_model.LinearRegression

m = model(**params)

In [ ]:
%%time
m.fit(to.train.xs.values, to.train.ys.values.ravel())

In [ ]:
%%time
y_valid_pred = m.predict(to.valid.xs.values)

In [ ]:
%%time
if do_test:
    y_test_pred = m.predict(test_dl.xs)

In [ ]:
y_valid_true = to.valid.ys.values.ravel()
nb_score = evaluate_torch(torch.from_numpy(y_valid_true), 
                          torch.from_numpy(y_valid_pred)).item()
print(f'sklearn loss {nb_score:.4f}')

### `fastai`

Fastai finding: make sure your test set values are not out of domain $\Rightarrow$ `timestampYear` in this notebook is put into the training set but there only takes on the value 2016.0, but in the test set it's 2017.0 and 2018.0, causing the predictions to zero out everywhere.

In [ ]:
y_range = [0,
           np.max([to.train.ys.values.max(), 
                   to.valid.ys.values.max()]),]
y_range

In [ ]:
#export
class Swish(nn.ReLU):
    def forward(self, input:Tensor) -> Tensor:
        if self.inplace:
            res = input.clone()
            torch.sigmoid_(res)
            input *= res
            return input
        else:
            return torch.sigmoid(input) * input
    
class Sine(nn.ReLU):
    def forward(self, input:Tensor) -> Tensor:
        if self.inplace:
            return torch.sin_(input)
        else:
            return torch.sin(input)

In [ ]:
layers = [500, 250, 125]

# config = None
config = tabular_config(embed_p=.1, ps = [.1, .1, .1])
# config = tabular_config(act_cls=nn.ReLU(inplace=True))
# config = tabular_config(act_cls=Swish(inplace=True))
# config = tabular_config(act_cls=Sine(inplace=True))

learn = tabular_learner(dls, y_range=y_range, layers=layers,
                        n_out=1, config=config, loss_func=evaluate_torch)

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(5, lr_max=1e-2)

In [ ]:
learn.recorder.plot_loss()

In [ ]:
%%time
y_valid_pred, y_valid_true = learn.get_preds()

In [ ]:
%%time
if do_test:
    y_test_pred, _ = learn.get_preds(dl=test_dl)
    y_test_pred = cnr(y_test_pred)

In [ ]:
nb_score = evaluate_torch(y_valid_true, 
                          y_valid_pred).item()
print(f'fastai loss {nb_score:.4f}')

In [ ]:
y_valid_pred, y_valid_true = cnr(y_valid_pred), cnr(y_valid_true)

## `lightgbm`

In [ ]:
%%time
lgb_train = lgb.Dataset(to.train.xs.values, to.train.ys.values.ravel())
lgb_eval = lgb.Dataset(to.valid.xs.values, to.valid.ys.values.ravel(), 
                       reference=lgb_train)

In [ ]:
# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'l2',
    'num_leaves': 42,
    'learning_rate': 0.5,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

In [ ]:
%%time
gbm = lgb.train(params, lgb_train,
                num_boost_round=100,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

In [ ]:
%%time
y_valid_pred = gbm.predict(to.valid.xs.values,
                           num_iteration=gbm.best_iteration)

In [ ]:
%%time
if do_test:
    y_test_pred = gbm.predict(test_dl.xs.values,
                              num_iteration=gbm.best_iteration)

In [ ]:
y_valid_true = to.valid.ys.values.ravel()
nb_score = evaluate_torch(torch.from_numpy(y_valid_true), 
                          torch.from_numpy(y_valid_pred)).item()
print(f'sklearn loss {nb_score:.4f}')

## Inspecting

### `dep_var` distribution

Train vs validation distributions

In [ ]:
#export
pick_random = lambda x: np.random.choice(x, size=5000, replace=False)

In [ ]:
preprocessing.hist_plot_preds(pick_random(y_valid_true), 
                              pick_random(y_valid_pred), 
                              label0='truth', label1='prediction')

In [ ]:
if do_test:
    preprocessing.hist_plot_preds(pick_random(y_valid_true), 
                                  pick_random(y_test_pred), 
                                  label0='truth (validation)', 
                                  label1='prediction (test set)').show()

### Boldly wrong predictions

In [ ]:
%%time
bwt = preprocessing.BoldlyWrongTimeseries(to.valid.xs, y_valid_true, y_valid_pred,
                                          t=df.iloc[splits[1]].loc[:,['timestampElapsed']].copy())

In [ ]:
bwt.run_boldly()

Finding:
- lgbm makes predictions of negative values!

## Submission to kaggle

In [ ]:
%%time
if do_test:
    y_test_pred_original = torch.exp(tensor(y_test_pred)) - 1

    y_out = pd.DataFrame(cnr(y_test_pred_original),
                         columns=['meter_reading'],
                         index=df_test.index)
    display(y_out.head())

    assert len(y_out) == 41697600

In [ ]:
%%time
if do_submit:
    y_out.to_csv(data_path/'my_submission.csv')

`kaggle competitions submit -c ashrae-energy-prediction -f submission.csv -m "Message"`

In [ ]:
# message = ['random forest', '500 obs/bid', 'all features', f'nb score {nb_score:.4f}']
message = ['lightgbm', '500 obs/bid', '100 rounds', '42 leaves', 'lr .5', f'nb score {nb_score:.4f}']
# message = ['tabular_learner', '500 obs/bid', 'all features', f'layers {layers}, embed_p .1, ps [.1,.1,.1]', f'nb score {nb_score:.4f}']
message = ' + '.join(message)
message

In [ ]:
if do_test and do_submit:
    print('Submitting...')
    !kaggle competitions submit -c ashrae-energy-prediction -f '{data_path}/my_submission.csv' -m '{message}'

**submission scores**

random forest:
- 5 obs per building ID, .75 max_features, 100 estimators: 
    - nb score = 2.37
    - kaggle score = 1.68 / 1.86
    
tabular learner:
- 5 obs per building ID, layers=[500,250], lr = 2e-3: 
    - nb score = 1.55
    - kaggle score = 1.8 / 2.13
- 5 obs per building ID, layers=[500,250], second run with lr = 1e-3: 
    - nb score = 1.57
    - kaggle score = 1.846 / 2.13
- 50 obs per building ID, layers=[500,250], 2 rounds: 
    - nb score = 1.39
    - kaggle score = 1.722 / 2.51
- 50 obs per building ID, layers=[500,250], 2 rounds: 
    - nb score = 1.34
    - kaggle score = 1.641 / 2.266
- 50 obs per building ID, layers=[500,250], 2 rounds, bs=256: 
    - nb score = 1.32
    - kaggle score = 1.643 / 1.926
- 500 obs per building ID, layers=[500,250], 3 rounds: 
    - nb score = 1.19
    - kaggle score = 1.62 / 2.55

Finding:
- nb scores are lower than the kaggle scores
- random forest seems to have public and private score closer to each other than tabular learner

**randomly splitting**
    
Finding (modified target values, all info = info except time):
- Linear:
    - meter only @100k: 2.1
    - all info minus time @100k: 2.3
    - all info incl time @100k: 2.32
    - all info incl time + ids @100k: 2.32
- RandomForest:
    - meter only @100k: 2.2
    - all info minus time @100k: 2.7
    - all info incl time @100k: 2.74
    - all info incl time + ids @100k: 2.82
- tabular_learner:
    - meter only @100k: 2.1
    - all info minus time @100k: 1.56
    - all info incl time @100k: 1.52
    - all info incl time + ids @100k: 0.96
    
**splitting along time**
Finding:
- Linear:
    - meter only @100k: 2.1
    - all info minus time @100K: 2.2
    - all info incl time @100k: 2.3
    - all info incl time + ids @100k: 2.29
- RandomForest:
    - meter only @100k: 2.1
    - all info minus time @100K: 2.7
    - all info incl time @100k: 2.52
    - all info incl time + ids @100k: 2.62
- tabular_learner:
    - meter only @100k: 2.06
    - all info minus time @100K: 1.62
    - all info incl time @100k: 1.62
    - all info incl time + ids @100k: 1.31